In [1]:
import math
import random
import cmath
import math
import numpy as np
from collections import deque
import hashlib
import struct
import os 


In [2]:


def load_vector_from_file(filename):
    with open(filename, 'rb') as file:
        # Read the size of the vector (stored as size_t in C++, which is platform-dependent)
        size_data = file.read(struct.calcsize('P'))  
        size = struct.unpack('P', size_data)[0]
        
        # Read the rest of the data as uint64_t values
        vector_data = file.read(size * struct.calcsize('Q')) 
        vector = struct.unpack(f'{size}Q', vector_data)
        
        return list(vector)

In [3]:
def does_gm_minimizer_exists(w, k):
    return os.path.exists(f'gm orders/w{w}_k{k}.bin')

def get_gm_minimizer_dna_lex(w,k):
    # Load order
    order = load_vector_from_file(f'gm orders/w{w}_k{k}.bin')

    # get max value
    max_value = 2**k
    rank = max_value
    for i in range(len(order)):
        if order[i] == max_value:
            order[i] = rank
            rank += 1
        



    def optimal_minimizer_dna(kmer, w, k, seed):
        original_kmer = kmer  # Save the original kmer for printing
        odd_bits = 0
        even_bits = 0
        bit_position = 0

        for position in range (2*k):
            current_bit = kmer & 1
            if position % 2 == 0:
                even_bits = (current_bit << (position//2)) | even_bits
            else:
                odd_bits = (current_bit << (position//2)) | odd_bits
            kmer >>= 1

        odd_bits_shifted = odd_bits 
        return ((order[odd_bits_shifted] * 2**k)+ even_bits)

    return optimal_minimizer_dna

In [4]:
def random_density_function_mod_minimizer_dna(w, k, function, n_samples=10000, seed = 1, r = 4):

    t = r + ((k-r) % w)

    t_mask = (1 << (2*t)) - 1

    tmer = 0
    rank_window = deque()

    max_rank_widow_size = w + k -t
    last_sampled_pos = 0


    # generate initial random kmer
    for _ in range(k):
        bits = random.getrandbits(2)
        tmer = ((tmer << 2) | bits) & t_mask
    
    # generate initial rank window
    for _ in range(max_rank_widow_size):
        next_bits = random.getrandbits(2)
        tmer = ((tmer << 2) | next_bits) & t_mask
        rank = function(tmer, w+k-t, t, seed)
        rank_window.append(rank)

    # iterate over all samples
    sampled_positions = 0
    for _ in range(0, n_samples):

        # update rank widow
        rank_window.popleft()
        next_bits = random.getrandbits(2)
        tmer = ((tmer << 2) | next_bits) & t_mask
        suffix_rank = function(tmer, w+k-t, t,seed)
        rank_window.append(suffix_rank)

        # Get new sampled position
        sampled_pos = rank_window.index(min(rank_window))
        sampled_pos = sampled_pos % w

        # position sampled last is now 1 behind
        last_sampled_pos -= 1

        # check if sampled a new position
        if sampled_pos != last_sampled_pos:
            sampled_positions += 1
            last_sampled_pos = sampled_pos


    total_number_of_kmers = n_samples + w -1 #first sample has w kmers
    density = sampled_positions / total_number_of_kmers
    return density

In [5]:
def mm_gm_density_dna(w,k,n_samples):
    r = 4
    t = r + ((k - r) % w)  # t calculated based on k and w
    w_adjusted = w + k - t
    w_adjusted = min(w_adjusted,15)
    if does_gm_minimizer_exists(w_adjusted, t):
        density = random_density_function_mod_minimizer_dna(
            w=w, k=k, function=get_gm_minimizer_dna_lex(w_adjusted, t), n_samples=n_samples, seed=0, r=r)
        return density
    else:
        return np.nan

Kraken

In [6]:
w = 17
k = 15


mm_gm_density_dna(w, k, 100000)

# can try generate a minimizer for k = 15 w= 17

0.0880359142537194

Kraken 2

In [7]:
w = 5
k = 31

mm_gm_density_dna(w,k, 100000)

0.22231110755569777

KMC2

In [8]:
w = 22
k = 7
mm_gm_density_dna(w, k, 100000)

0.07382449685566031

In [9]:
w = 49
k = 7
mm_gm_density_dna(w, k, 100000)

0.036172637134175595

KMC3

In [10]:
w = 16
k = 9
mm_gm_density_dna(w, k, 100000)

0.09554566814977754

SSHash

In [11]:
w = 11
k = 21

mm_gm_density_dna(w,k, 100000)

0.12005799420057994

Minimap1/2 / MetaProb2

In [12]:
w = 10
k = 15
mm_gm_density_dna(w, k, 1000000)

0.13275880517075347

Giraffe

In [13]:
w = 11
k = 29

mm_gm_density_dna(w,k, 100000)

0.11226877312268774

GraphAligner

In [17]:
w = 30
k = 19

mm_gm_density_dna(w,k, 100000)

nan